This notebook contains the code that accompanies my blog on PySpark. The blog post is located [here](https://zach-a-greenberg.medium.com/apache-spark-with-pyspark-60b01bcc089)

For this demo, I am going to use a dataset I created via webscrapping (see this [blog](https://zach-a-greenberg.medium.com/web-scraping-with-python-using-beautifulsoup-d8472555fcb5)). While this dataset is not large and I am not actually connected to the cloud, the same coding principles apply. 

In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession, functions as F

In order to use Spark, we need to establish a connection. The connection is made in the cells below. 

In [2]:
#creating a session
spark = SparkSession.builder.getOrCreate()

Spark has it's own way of reading in the data. The parameter header=True makes the assumption that the first row contains the column names.

In [3]:
#reading in the DataFrame with Spark
broadway = spark.read.csv('broadway.csv', header=True)

In [4]:
broadway.show(5)

+---+--------------------+--------------------+--------------------+-------------------+
|_c0|               Shows|         Description|             Theater|Current Ticket Cost|
+---+--------------------+--------------------+--------------------+-------------------+
|  0|             Wicked |Meet the witches ...|    Gershwin Theatre|              89.00|
|  1|Moulin Rouge! The...|A theatrical cele...|Al Hirschfeld The...|              69.00|
|  2|      The Lion King |Pride Rock comes ...|    Minskoff Theatre|              75.00|
|  3|To Kill a Mocking...|Harper Lee’s clas...|     Shubert Theatre|              29.00|
|  4|           Hamilton |A fresh look at t...|Richard Rodgers T...|             149.00|
+---+--------------------+--------------------+--------------------+-------------------+
only showing top 5 rows



In the cell below we are dropping the 'index column' from the original dataset. Just like in Pandas, we can do this inplace by overwriting the variable.

In [5]:
#dropping the _c0 or index column
broadway = broadway.drop('_c0')

In Spark, the printSchema method is the way to check the datatypes. We can also check to see if null values are allowed in the data from the nullable argument. m

In [6]:
#checking the datatypes
broadway.printSchema()

root
 |-- Shows: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Theater: string (nullable = true)
 |-- Current Ticket Cost: string (nullable = true)



When changing a datatype in Spark, we are essentially overwriting a column and using the cast method to make the datatype conversion. Double is Spark's version of a float. 

In [7]:
#changing the cost datatype to a 'double' which is the equivalent of a decimal
broadway = broadway.withColumn('Current Ticket Cost', broadway['Current Ticket Cost'].cast('double'))

When formatting text columns, similar principles apply. In a few steps we can modify the text using the functions provided in the pyspark.sql module. A list of them can be found [here](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/sql/functions.html)

In [8]:
#dropping the word 'Theatre' from all of the entries in the Theater column

#we create a column to perform the split of text
broadway = broadway.withColumn('Split', F.split(broadway.Theater, 'Theatre'))

#we overwrite the original column with the desired text part
broadway = broadway.withColumn('Theater', broadway.Split.getItem(0))

#we drop the column used to make the split
broadway = broadway.drop('Split')

broadway.show(3)

+--------------------+--------------------+--------------+-------------------+
|               Shows|         Description|       Theater|Current Ticket Cost|
+--------------------+--------------------+--------------+-------------------+
|             Wicked |Meet the witches ...|     Gershwin |               89.0|
|Moulin Rouge! The...|A theatrical cele...|Al Hirschfeld |               69.0|
|      The Lion King |Pride Rock comes ...|     Minskoff |               75.0|
+--------------------+--------------------+--------------+-------------------+
only showing top 3 rows



THIS! A parquet is a special Spark DataFrame that allows use to quickly analyze the data. It is wise to create a parquet version of a DataFrame. 

In [9]:
#in order of make a parquet, the columns CANNOT have any spaces in them so I will rename them
broadway = broadway.withColumnRenamed('Current Ticket Cost', 'Cost')

#rewriting the dataframe as a parquet, a Spark specific file enabling for faster analysis
broadway.write.parquet('broadway.parquet', mode = 'overwrite')

In [10]:
shows = spark.read.parquet('broadway.parquet')

In [11]:
shows.printSchema()

root
 |-- Shows: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Theater: string (nullable = true)
 |-- Cost: double (nullable = true)



Now that we have a parquet, we can perform some simple analysis. Just like Pandas, we can use the describe method to see some summary stats.

In [12]:
#getting the summary stats of all the columns, the only one we really care about is Cost
shows.describe().show()

+-------+--------------------+--------------------+--------------+-----------------+
|summary|               Shows|         Description|       Theater|             Cost|
+-------+--------------------+--------------------+--------------+-----------------+
|  count|                  24|                  24|            24|               24|
|   mean|                null|                null|          null|64.20833333333333|
| stddev|                null|                null|          null|26.12466628314699|
|    min|Ain't Too Proud –...|A deeply personal...|Al Hirschfeld |             29.0|
|    max|             Wicked |When the world st...|Winter Garden |            149.0|
+-------+--------------------+--------------------+--------------+-----------------+



We can put the parquet into a SQL table to write SQL queries for analysis.

In [13]:
#temporarily making the parquet into a SQL table for querying
shows.createOrReplaceTempView('top25')

In [14]:
#writing a SQL query for the top 3 most expensive shows
spark.sql('SELECT Shows, Cost FROM top25 WHERE Cost > 65.00 ORDER BY Cost DESC LIMIT 3').collect()

[Row(Shows='Hamilton ', Cost=149.0),
 Row(Shows='The Music Man ', Cost=99.0),
 Row(Shows='Wicked ', Cost=89.0)]

We can also use the filter method on the parquet to filter the data to find specific information.

In [15]:
#using the filtering method to find shows that cost less than $60
shows['Shows', 'Cost'].filter(shows.Cost < 60).show()

+--------------------+----+
|               Shows|Cost|
+--------------------+----+
|To Kill a Mocking...|29.0|
|            Aladdin |57.5|
|          Hadestown |49.0|
|            Chicago |49.5|
|The Phantom of th...|29.0|
|     Come From Away |49.0|
|David Byrne's Ame...|59.0|
|Ain't Too Proud –...|39.0|
|     Mrs. Doubtfire |49.0|
| Jagged Little Pill |49.0|
|            Company |59.0|
|Girl From the Nor...|39.0|
|     MJ The Musical |59.0|
|              Diana |49.0|
+--------------------+----+

